In [1]:
import os
import sys

import numpy as np

from ase.calculators.lammpsrun import LAMMPS
from ase.io import read

import catkit
from catkit.gen.adsorption import get_adsorption_sites

from mcmc import mcmc_run, get_adsorption_coords

from htvs.djangochem.pgmols.utils import surfaces

In [2]:
os.environ["LAMMPS_COMMAND"]="/home/dux/lammps/src/lmp_serial"
os.environ["LAMMPS_POTENTIALS"]="/home/dux/lammps/potentials/"

In [3]:
from time import perf_counter

start = perf_counter()
num_sweeps = 100

# use LAMMPS
parameters = {
    'pair_style': 'eam',
    'pair_coeff': ['* * Au_u3.eam']
}

potential_file = os.path.join(os.environ["LAMMPS_POTENTIALS"], 'Au_u3.eam')
lammps_calc = LAMMPS(files=[potential_file], keep_tmp_files=False, keep_alive=True, tmp_dir="/home/dux/surface_sampling/tmp_files")
lammps_calc.set(**parameters)

# Au from standard cell
atoms = read('Au_mp-81_conventional_standard.cif')
slab, surface_atoms = surfaces.surface_from_bulk(atoms, [1,1,0], size=[4,4])
slab.write("Au_110_pristine_slab.cif")

coords, connectivity, sym_idx = get_adsorption_sites(slab, symmetry_reduced=False)

element = 'Au'
metal = catkit.gratoms.Gratoms(element)

ads_coords = get_adsorption_coords(slab, metal, connectivity)

/home/dux/miniconda3/envs/surface_sampling/lib/python3.8/site-packages/catkit/gen/adsorption.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r1_topology = np.array(self.r1_topology)
/home/dux/miniconda3/envs/surface_sampling/lib/python3.8/site-packages/catkit/gen/adsorption.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r2_topology = np.array(self.r2_topology)


In [5]:
all_adsorbed = read("Au_110_all_adsorbed_slab.cif")
assert len(all_adsorbed) == len(slab) + len(ads_coords)

AssertionError: 

In [8]:
len(slab)

112

In [9]:
len(all_adsorbed)

256

Manually removed some atoms

In [ ]:
proper_adsorbed = read("Au_110_proper_adsorbed_slab.cif")
ads_positions = proper_adsorbed.get_positions()[len(slab):]

In [ ]:
ads_positions.shape

In [ ]:
len(ads_positions)

In [7]:
slab, surface_atoms = surfaces.surface_from_bulk(atoms, [1,1,0], size=[4,4])

element = 'Au'
num_ads_atoms = 16 + 8 # for canonical runs
alpha = 0.99
num_sweeps = 300

history, energy_hist, frac_accept_hist, adsorption_count_hist = mcmc_run(num_sweeps=num_sweeps, temp=1, pot=0, alpha=alpha, slab=slab, calc=lammps_calc, element=element, canonical=True, num_ads_atoms=num_ads_atoms, ads_coords=None)

INFO:Running with num_sweeps = 300, temp = 1, pot = 0, alpha = 0.99
INFO:there are 28 atoms 
INFO:using slab calc <ase.calculators.lammpsrun.LAMMPS object at 0x7fe7c9e7fb20>
/home/dux/miniconda3/envs/surface_sampling/lib/python3.8/site-packages/catkit/gen/adsorption.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r1_topology = np.array(self.r1_topology)
/home/dux/miniconda3/envs/surface_sampling/lib/python3.8/site-packages/catkit/gen/adsorption.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.r2_topology = np.array(self.r2_topology

FileNotFoundError: [Errno 2] No such file or directory: 'lammps'

In [ ]:
import matplotlib.pyplot as plt
runs = range(1, num_sweeps+1)
# do the plots
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
ax[0, 0].plot(runs, energy_hist)
ax[0, 0].set_xlabel("Iter")
ax[0, 0].set_ylabel("Energy (E)")
ax[0, 0].set_title("Energy (E) vs Sweeps")

ax[0, 1].plot(runs, frac_accept_hist)
ax[0, 1].set_xlabel("Iter")
ax[0, 1].set_ylabel("Fraction accepted")
ax[0, 1].set_title("Fraction accepted vs Sweeps")

ax[1, 1].plot(runs, np.array(list(adsorption_count_hist.values())).T)
ax[1, 1].set_xlabel("Iter")
ax[1, 1].set_ylabel("Adsorption count")
ax[1, 1].legend(adsorption_count_hist.keys())
ax[1, 1].set_title("Adsorption count vs Iterations")

fig.show()
fig.tight_layout()

In [ ]:
from mcmc import mcmc_run, slab_energy

# calculated energy of desorbed rows (2 adjacent rows removed)
aligned_sites = read("Au_canonical_aligned_sites_300_iter.cif")
aligned_sites.calc = lammps_calc
print(f"aligned sites slab energy is {slab_energy(aligned_sites)}")

# non aligned sites (a lot more possibilites)
nonaligned_sites = read("Au_canonical_nonaligned_sites_300_iter.cif")
nonaligned_sites.calc = lammps_calc
print(f"nonaligned sites slab energy is {slab_energy(nonaligned_sites)}")


Try with relaxation

In [ ]:
slab, surface_atoms = surfaces.surface_from_bulk(atoms, [1,1,0], size=[4,4])

element = 'Au'
num_ads_atoms = 16 + 8 # for canonical runs
alpha = 0.99
num_runs = 300

history, energy_hist, frac_accept_hist, adsorption_count_hist = mcmc_run(num_runs=num_runs, temp=1, pot=0, alpha=alpha, slab=slab, calc=lammps_calc, element=element, canonical=True, num_ads_atoms=num_ads_atoms, ads_coords=ads_positions, relax=True)

In [ ]:
# slab, surface_atoms = surfaces.surface_from_bulk(atoms, [1,1,0], size=[4,4])
# slab.write("Au_110_starting_slab.cif")